In [1]:
! pip install datasets transformers[torch] 'accelerate>=0.26.0' peft bitsandbytes

In [3]:
from datasets import load_dataset


def load_io_dataset(split, uf_n=40):
    dataset = load_dataset("json", data_files=f"TTE_uf{uf_n}/TTE_with_IO_{split}.json")

    def format_example(example):
        return {
            "text": f"Prompt: {example['input']}\nResponse: {example['output']}\n"
        }

    dataset = dataset["train"].map(format_example)
    return dataset

train_dataset = load_io_dataset("train")
test_dataset = load_io_dataset("test")
eval_dataset = load_io_dataset("eval")
print(train_dataset)
print(test_dataset)
print(eval_dataset)

FileNotFoundError: Unable to find '/home/jovyan/Compression_evaluation/TTE_uf40/TTE_with_IO_train.json'

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


#model_id = "../LA-Framework/results/facebook-350m/train/sft/e1-bs2-lr0.0005-G1-N1-NN1/5717"

#model = AutoModelForCausalLM.from_pretrained(
#    model_id
#)
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#tokenizer.pad_token = tokenizer.eos_token  # Set pad token for batching


model_name = "../Qwen/models--Qwen--Qwen3-4B/snapshots/1cfa9a7208912126459214e8b04321603b3df60c"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=1024)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = test_dataset.map(tokenize_function, batched=True)  # dataset used in the PEFT for eval
#tokenized_eval = eval_dataset.map(tokenize_function, batched=True)  # dataset used to eval the resulting LLM

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

Map:   0%|          | 0/4430 [00:00<?, ? examples/s]

Map:   0%|          | 0/4430 [00:00<?, ? examples/s]

In [12]:
prompt = eval_dataset[1]['input']
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=600)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Based on the user's watching history and the film's rating, order the 50 candidate films. Order the films where the first one of the list is the most likely to be watched by the user. The output should only contain the ordered list of the recommended films bounded by the  special strings '%% START RECOMMENDED LIST %%' and '%% END LIST %%'. Here follows the user's history and the list of candidate films.
%% START HISTORY %%
Movie name: Payback (1999)	Rating:3
Movie name: Under Siege (1992)	Rating:3
Movie name: Independence Day (ID4) (1996)	Rating:3
Movie name: Double Jeopardy (1999)	Rating:3
Movie name: Maverick (1994)	Rating:4
Movie name: Man in the Iron Mask, The (1998)	Rating:3
Movie name: Die Hard: With a Vengeance (1995)	Rating:3
Movie name: Getaway, The (1994)	Rating:3
Movie name: Demolition Man (1993)	Rating:3
Movie name: Conspiracy Theory (1997)	Rating:3
%% END HISTORY %%
%% START CANDIDATES %%
Movie name: Them! (1954)
Movie name: Penitentiary II (1982)
Movie name: Life and Time

In [4]:
print(train_dataset[0]['text'])

Prompt: Based on the user's watching history and the film's rating, order the 50 candidate films. Order the films where the first one of the list is the most likely to be watched by the user. The output should only contain the ordered list of the recommended films bounded by the  special strings '%% START RECOMMENDED LIST %%' and '%% END LIST %%'. Here follows the user's history and the list of candidate films.
%% START HISTORY %%
Movie name: Fargo (1996)	Rating:4
Movie name: Run Lola Run (Lola rennt) (1998)	Rating:4
Movie name: Rain Man (1988)	Rating:5
Movie name: Saving Private Ryan (1998)	Rating:5
Movie name: Awakenings (1990)	Rating:5
Movie name: Gigi (1958)	Rating:4
Movie name: Sound of Music, The (1965)	Rating:5
Movie name: Driving Miss Daisy (1989)	Rating:4
Movie name: Bambi (1942)	Rating:4
Movie name: Mary Poppins (1964)	Rating:5
%% END HISTORY %%
%% START CANDIDATES %%
Movie name: I'm Not Rappaport (1996)
Movie name: Steamboat Willie (1940)
Movie name: Run Silent, Run Deep (19

In [5]:
print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layer

In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for quantized training
#model = prepare_model_for_kbit_training(model)

# LoRA config: Target q_proj and v_proj in attention layers

# facebook/OPT-350m config
lora_config = LoraConfig(
    r=16,  # Rank of adapters (higher = more params, better adaptation; best 8-32)
    lora_alpha=32,  # Scaling factor
    target_modules=["k_proj", "v_proj", "q_proj", "out_proj"],  # Modules to adapt (based on model architecture)
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


# Qwen3-4B configuration
lora_config = LoraConfig(
    r=32,                     # or 64 if you have ≥16–20 GB VRAM free during training
    lora_alpha=64,            # classic 2×r rule — very stable on Qwen models
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",     # attention
        "gate_proj", "up_proj", "down_proj"         # MLP — almost always worth including on Qwen
    ],
    lora_dropout=0.0,         # 0 is fine & faster with modern impl (Unsloth / transformers ≥4.40)
    bias="none",
    task_type="CAUSAL_LM",
    # Optional but sometimes helpful:
    # use_rslora=True,        # rank-stabilized LoRA — can help when r is high
    # loftq_config=None,      # only if you're doing LoftQ initialization
)
# Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()  # Shows trainable params


trainable params: 66,060,288 || all params: 4,088,528,384 || trainable%: 1.6157


In [15]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from transformers import Trainer

"""
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    # num_train_epochs=3,
    num_train_epochs=2,
    per_device_train_batch_size=4,  # Based on GPU memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Effective batch size = batch_size * accum_steps
    learning_rate=2e-4,
    fp16=True,  # Mixed precision for speed
    save_steps=100,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)
"""

# Qwen TA configuration

training_args = TrainingArguments(
    output_dir="./fine_tuned_qwen3-4b",

    num_train_epochs=1,                     # ← changed: most Qwen3/2.5 LoRA runs use 1 epoch nowadays
    max_steps=-1,                           # use epochs instead unless you want fixed steps

    per_device_train_batch_size=2,          # ↓ safer starting point for 4B model on 16–24 GB
    per_device_eval_batch_size=4,           # eval can usually be larger
    gradient_accumulation_steps=8,          # → effective batch size ≈ 16 (sweet spot for LoRA)

    learning_rate=1e-4,                     # ↓ most recommended now (2e-4 often too aggressive → instability/overfit)
    lr_scheduler_type="cosine",             # ← changed: smoother decay, very popular on Qwen & modern runs
    #warmup_ratio=0.03,                      # or warmup_steps=100–300 depending on total steps
    warmup_steps=100,
    weight_decay=0.01,                      # small regularization helps

    #optim="adamw_8bit",                     # ← changed: very memory efficient & stable (needs bitsandbytes)
    # optim="paged_adamw_8bit"              # if you see CPU offloading needed

    fp16=False,                             # ← usually disable if you can
    bf16=True,                              # ← changed: strongly preferred on Ampere+ GPUs (A100, RTX 30/40/50xx)
    # tf32=True                             # optional on A100/H100

    gradient_checkpointing=True,            # very important for 4B+ models → saves ~30–40% VRAM
    gradient_checkpointing_kwargs={"use_reentrant": False},  # modern & faster (transformers ≥4.36)

    max_grad_norm=1.0,                      # helps stability on LoRA

    logging_steps=20,                       # more frequent than 10 → better progress visibility
    save_steps=200,                         # less spam than 100
    save_strategy="steps",
    save_total_limit=2,                     # keep only last 2 checkpoints

    eval_strategy="steps",
    eval_steps=200,                         # match save_steps or slightly more frequent
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    report_to="none",                       # or "wandb"/"tensorboard" if you track
    # ddp_find_unused_parameters=False      # sometimes needed in multi-GPU
)


# Data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

In [16]:
trainer.train()

# Evaluate
results = trainer.evaluate()
print(results)  # Shows eval_loss, etc.
peft_model.save_pretrained("./fine_tuned_adapters_qwen3-4b")

Step,Training Loss,Validation Loss
200,0.508159,0.509506


{'eval_loss': 0.5095056295394897, 'eval_runtime': 351.2432, 'eval_samples_per_second': 12.612, 'eval_steps_per_second': 3.155, 'epoch': 1.0}


In [4]:
# Save PEFT adapters
#peft_model.save_pretrained("./fine_tuned_adapters")

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM

model_name = "../Qwen/models--Qwen--Qwen3-4B/snapshots/1cfa9a7208912126459214e8b04321603b3df60c"

inference_model = AutoPeftModelForCausalLM.from_pretrained(
    "./fine_tuned_adapters_qwen3-4b",
    device_map="cuda",
    #torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.12/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized

In [18]:
# prompt = "Based on the user's watching history and the film's rating, select 10 films from the 50 candidates. Order the 10 selected films where the first one is the most likely to be watched by the user. The output should only contain the ordered list of the selected films. Here follows the user's history and the list of candidate films.\n%% START HISTORY %%\nMovie name: Toy Story 2 (1999)\tRating:4\nMovie name: Airplane! (1980)\tRating:4\nMovie name: Pleasantville (1998)\tRating:3\nMovie name: Dumbo (1941)\tRating:5\nMovie name: Princess Bride, The (1987)\tRating:3\nMovie name: Snow White and the Seven Dwarfs (1937)\tRating:4\nMovie name: Miracle on 34th Street (1947)\tRating:4\nMovie name: Ponette (1996)\tRating:4\nMovie name: Schindler's List (1993)\tRating:5\nMovie name: Aladdin (1992)\tRating:4\n%% END HISTORY %%\n%% START CANDIDATES %%\nMovie name: Cowboy Way, The (1994)\nMovie name: Miller's Crossing (1990)\nMovie name: Slumber Party Massacre, The (1982)\nMovie name: Nashville (1975)\nMovie name: Shanghai Noon (2000)\nMovie name: Mad Max 2 (a.k.a. The Road Warrior) (1981)\nMovie name: Deliverance (1972)\nMovie name: Never Talk to Strangers (1995)\nMovie name: Naked in New York (1994)\nMovie name: Tarzan (1999)\nMovie name: Popeye (1980)\nMovie name: Red Corner (1997)\nMovie name: Hudsucker Proxy, The (1994)\nMovie name: Live and Let Die (1973)\nMovie name: Cutthroat Island (1995)\nMovie name: Dead Presidents (1995)\nMovie name: Seven (Se7en) (1995)\nMovie name: Wirey Spindell (1999)\nMovie name: Eyes of Laura Mars (1978)\nMovie name: Fisher King, The (1991)\nMovie name: Hackers (1995)\nMovie name: Brother Minister: The Assassination of Malcolm X (1994)\nMovie name: Hunchback of Notre Dame, The (1996)\nMovie name: Man Who Knew Too Much, The (1934)\nMovie name: Hideaway (1995)\nMovie name: Mr. Death: The Rise and Fall of Fred A. Leuchter Jr. (1999)\nMovie name: Penitentiary II (1982)\nMovie name: Crocodile Dundee II (1988)\nMovie name: Pocahontas (1995)\nMovie name: Friends & Lovers (1999)\nMovie name: Bug's Life, A (1998)\nMovie name: Under the Rainbow (1981)\nMovie name: Antz (1998)\nMovie name: Gandhi (1982)\nMovie name: Carrie (1976)\nMovie name: Professional, The (a.k.a. Leon: The Professional) (1994)\nMovie name: Close Shave, A (1995)\nMovie name: Hercules (1997)\nMovie name: Beauty and the Beast (1991)\nMovie name: Toy Story (1995)\nMovie name: Mulan (1998)\nMovie name: Shopping (1994)\nMovie name: Matilda (1996)\nMovie name: Rosencrantz and Guildenstern Are Dead (1990)\nMovie name: Muppet Treasure Island (1996)\nMovie name: Commandments (1997)\nMovie name: Rawhead Rex (1986)\nMovie name: Mulholland Falls (1996)\nMovie name: Insomnia (1997)\nMovie name: Harlem (1993)\n%% END CANDIDATES %%"
prompt = eval_dataset[1]['input']
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = inference_model.generate(**inputs, max_new_tokens=600)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# prompt = "Based on the user's watching history and the film's rating, select 10 films from the 50 candidates. Order the 10 selected films where the first one is the most likely to be watched by the user. The output should only contain the ordered list of the selected films. Here follows the user's history and the list of candidate films.\n%% START HISTORY %%\nMovie name: Toy Story 2 (1999)\tRating:4\nMovie name: Airplane! (1980)\tRating:4\nMovie name: Pleasantville (1998)\tRating:3\nMovie name: Dumbo (1941)\tRating:5\nMovie name: Princess Bride, The (1987)\tRating:3\nMovie name: Snow White and the Seven Dwarfs (1937)\tRating:4\nMovie name: Miracle on 34th Street (1947)\tRating:4\nMovie name: Ponette (1996)\tRating:4\nMovie name: Schindler's List (1993)\tRating:5\nMovie name: Aladdin (1992)\tRating:4\n%% END HISTORY %%\n%% START CANDIDATES %%\nMovie name: Cowboy Way, The (1994)\nMovie name: Miller's Crossing (1990)\nMovie name: Slumber Party Massacre, The (1982)\nMovie name: Nashville (1975)\nMovie name: Shanghai Noon (2000)\nMovie name: Mad Max 2 (a.k.a. The Road Warrior) (1981)\nMovie name: Deliverance (1972)\nMovie name: Never Talk to Strangers (1995)\nMovie name: Naked in New York (1994)\nMovie name: Tarzan (1999)\nMovie name: Popeye (1980)\nMovie name: Red Corner (1997)\nMovie name: Hudsucker Proxy, The (1994)\nMovie name: Live and Let Die (1973)\nMovie name: Cutthroat Island (1995)\nMovie name: Dead Presidents (1995)\nMovie name: Seven (Se7en) (1995)\nMovie name: Wirey Spindell (1999)\nMovie name: Eyes of Laura Mars (1978)\nMovie name: Fisher King, The (1991)\nMovie name: Hackers (1995)\nMovie name: Brother Minister: The Assassination of Malcolm X (1994)\nMovie name: Hunchback of Notre Dame, The (1996)\nMovie name: Man Who Knew Too Much, The (1934)\nMovie name: Hideaway (1995)\nMovie name: Mr. Death: The Rise and Fall of Fred A. Leuchter Jr. (1999)\nMovie name: Penitentiary II (1982)\nMovie name: Crocodile Dundee II (1988)\nMovie name: Pocahontas (1995)\nMovie name: Friends & Lovers (1999)\nMovie name: Bug's Life, A (1998)\nMovie name: Under the Rainbow (1981)\nMovie name: Antz (1998)\nMovie name: Gandhi (1982)\nMovie name: Carrie (1976)\nMovie name: Professional, The (a.k.a. Leon: The Professional) (1994)\nMovie name: Close Shave, A (1995)\nMovie name: Hercules (1997)\nMovie name: Beauty and the Beast (1991)\nMovie name: Toy Story (1995)\nMovie name: Mulan (1998)\nMovie name: Shopping (1994)\nMovie name: Matilda (1996)\nMovie name: Rosencrantz and Guildenstern Are Dead (1990)\nMovie name: Muppet Treasure Island (1996)\nMovie name: Commandments (1997)\nMovie name: Rawhead Rex (1986)\nMovie name: Mulholland Falls (1996)\nMovie name: Insomnia (1997)\nMovie name: Harlem (1993)\n%% END CANDIDATES %%"
# prompt = eval_dataset[0]['input']
# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
# output = inference_model.generate(**inputs, max_new_tokens=400)
# print(tokenizer.decode(output[0], skip_special_tokens=True))

Based on the user's watching history and the film's rating, order the 50 candidate films. Order the films where the first one of the list is the most likely to be watched by the user. The output should only contain the ordered list of the recommended films bounded by the  special strings '%% START RECOMMENDED LIST %%' and '%% END LIST %%'. Here follows the user's history and the list of candidate films.
%% START HISTORY %%
Movie name: Payback (1999)	Rating:3
Movie name: Under Siege (1992)	Rating:3
Movie name: Independence Day (ID4) (1996)	Rating:3
Movie name: Double Jeopardy (1999)	Rating:3
Movie name: Maverick (1994)	Rating:4
Movie name: Man in the Iron Mask, The (1998)	Rating:3
Movie name: Die Hard: With a Vengeance (1995)	Rating:3
Movie name: Getaway, The (1994)	Rating:3
Movie name: Demolition Man (1993)	Rating:3
Movie name: Conspiracy Theory (1997)	Rating:3
%% END HISTORY %%
%% START CANDIDATES %%
Movie name: Them! (1954)
Movie name: Penitentiary II (1982)
Movie name: Life and Time

In [32]:
prompt = "Based on the user watching history and film's rating, rate the candidate films. Rate the candidate films from 1 to 10 where 1 represents a low rating and 10 represents high rating. The output should only contain the liast with the resulting rating. Here follows the user's history and the list of candidate films. %% START HISTORY %% Movie name: Silence of the Lambs, The (1991) Rating:6.0 Movie name: Risky Business (1983) Rating:8.0 Movie name: Program, The (1993) Rating:10.0 Movie name: Jurassic Park (1993) Rating:8.0 Movie name: Hand That Rocks the Cradle, The (1992) Rating:6.0 Movie name: Toy Story (1995) Rating:6.0 Movie name: Shawshank Redemption, The (1994) Rating:8.0 Movie name: American Beauty (1999) Rating:8.0 Movie name: Good Will Hunting (1997) Rating:10.0 Movie name: What's Eating Gilbert Grape (1993) Rating:2.0 Movie name: Searching for Bobby Fischer (1993) Rating:8.0 Movie name: Rudy (1993) Rating:10.0 Movie name: Few Good Men, A (1992) Rating:8.0 Movie name: Bug's Life, A (1998) Rating:6.0 Movie name: Happy Gilmore (1996) Rating:4.0 Movie name: Metropolitan (1990) Rating:8.0 Movie name: Varsity Blues (1999) Rating:10.0 Movie name: Forrest Gump (1994) Rating:10.0 Movie name: Ghost (1990) Rating:6.0 Movie name: There's Something About Mary (1998) Rating:6.0 %% END HISTORY %% %% START CANDIDATES %% Movie name: Grumpy Old Men (1993) Movie name: American Pie (1999) Movie name: Primary Colors (1998) Movie name: Mrs. Doubtfire (1993) Movie name: Die Hard 2 (1990) Movie name: Pleasantville (1998) Movie name: Inventing the Abbotts (1997) Movie name: How to Be a Player (1997) Movie name: Liar Liar (1997) Movie name: Sleepless in Seattle (1993) Movie name: Three Kings (1999) Movie name: Crimson Tide (1995) Movie name: Firm, The (1993) Movie name: River Runs Through It, A (1992) Movie name: Contact (1997) Movie name: Scent of a Woman (1992) Movie name: Primal Fear (1996) Movie name: Wedding Singer, The (1998) Movie name: Truman Show, The (1998) Movie name: Pretty Woman (1990) %% END CANDIDATES %%"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = inference_model.generate(**inputs, max_new_tokens=350)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Based on the user watching history and film's rating, rate the candidate films. Rate the candidate films from 1 to 10 where 1 represents a low rating and 10 represents high rating. The output should only contain the liast with the resulting rating. Here follows the user's history and the list of candidate films. %% START HISTORY %% Movie name: Silence of the Lambs, The (1991) Rating:6.0 Movie name: Risky Business (1983) Rating:8.0 Movie name: Program, The (1993) Rating:10.0 Movie name: Jurassic Park (1993) Rating:8.0 Movie name: Hand That Rocks the Cradle, The (1992) Rating:6.0 Movie name: Toy Story (1995) Rating:6.0 Movie name: Shawshank Redemption, The (1994) Rating:8.0 Movie name: American Beauty (1999) Rating:8.0 Movie name: Good Will Hunting (1997) Rating:10.0 Movie name: What's Eating Gilbert Grape (1993) Rating:2.0 Movie name: Searching for Bobby Fischer (1993) Rating:8.0 Movie name: Rudy (1993) Rating:10.0 Movie name: Few Good Men, A (1992) Rating:8.0 Movie name: Bug's Life, A 

In [68]:
# Serial LLM generation of evaluation's results

def split_list(text, start_token, end_token):
    phrase = text.split(start_token + "\n")
    movie_list = phrase[1].split("\n" + end_token)[0]
    movie_list = movie_list.split("\n")
    for i, movie in enumerate(movie_list):
        movie_list[i] = movie.split("\t")
    return movie_list


def perform_and_return_comparison(sample):
    prompt = sample['text']
    inputs = tokenizer(sample['input'], return_tensors="pt").to("cuda")
    output = inference_model.generate(**inputs, max_new_tokens=600)
    rating = tokenizer.decode(output[0], skip_special_tokens=True)
    
    rating = split_list(rating, "%% START RECOMMENDED LIST %%", "%% END LIST %%")
    return {'input': sample['input'],
            'rating': rating}

#print(eval_dataset[0])
#print(perform_and_return_comparison(eval_dataset[0]))
#eval_output = eval_dataset.map(perform_and_return_comparison)  # uncomment to perform serial output generation
#eval_output.to_json("eval_processed.json")                     # uncomment to perform serial output generation

Parameter 'function'=<function perform_and_return_comparison at 0x7f6a98596020> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.


Map:   0%|          | 0/4430 [00:00<?, ? examples/s]


KeyboardInterrupt


KeyboardInterrupt



In [ ]:
import torch
from tqdm.auto import tqdm   # optional, for nicer progress

# Parallel LLM generation of evaluation's results

def perform_and_return_comparison_batched(batch):
    # batch is a dict with lists: {'input': [...], 'text': [...]}
    inputs = tokenizer(
        batch['input'],
        return_tensors="pt",padding=True,
        truncation=True,
        max_length=1024,  # ← set sensible value, e.g. 2048
        padding_side='left'
    ).to("cuda")

    with torch.inference_mode():           # saves a bit of memory
        outputs = inference_model.generate(
            **inputs,
            max_new_tokens=600
        )

    decoded = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True
    )

    results = []
    for i, rating_full in enumerate(decoded):
        rating = split_list(
            rating_full,
            "%% START RECOMMENDED LIST %%",
            "%% END LIST %%"
        )
        results.append({
            'input': batch['input'][i],
            'rating': rating
        })

    return {'results': results}   # list of dicts — datasets will flatten it

# This is usually enough parallelism for one GPU
eval_output = eval_dataset.map(
    perform_and_return_comparison_batched,
    batched=True,
    batch_size=8,          # ← tune this: 4–16 most common sweet spot
                           # larger = better GPU util, but risk of OOM
    desc="Evaluating",
    remove_columns=eval_dataset.column_names,  # optional — keep only new fields
)

eval_output.to_json("eval_processed_parallel.json")

In [74]:
import pandas as pd

pd_data = pd.read_json(f"eval_processed.json", lines=True)
pd_data = pd_data.drop(columns=["input","output","text"])
print(pd_data.head(1))

def check_integrity(row, i):
    col = ["history", "candidates", "rating"]
    for c in col:
        for pair in row[c]:
            if len(pair) != 2:
                return -i
    return 0
    
def casting(list):
    new_list = []
    for obj in list:
        if (len(obj) == 2):
            [movie, rating] = obj
            new_pair = []
            new_pair.append(movie.removeprefix("Movie name: "))
            new_pair.append(float(rating.removeprefix("Rating:")))
            new_list.append(new_pair)
    return new_list

problems = []
for i in range(len(pd_data)):
    res = check_integrity(pd_data.iloc[i], i+1)
    if res < 0 :
        problems.append(-1-res)
print(problems)

#for column in pd_data.columns:
#    pd_data[column] = pd_data[column].apply(casting)
#print(pd_data.head(1))

                                             history  \
0  [[Movie name: She's Gotta Have It (1986), Rati...   

                                          candidates  \
0  [[Movie name: Sid and Nancy (1986), Rating:4.8...   

                                              rating  
0  [[Movie name: Sid and Nancy (1986), Rating:5.3...  
[4072, 8402, 8451, 8495, 9096, 10993, 15254]


In [73]:
for i in problems:
    for c in pd_data.columns:
        print(i, c, pd_data.iloc[i][c])

4072 history [['Movie name: Police Academy (1984)', 'Rating:8.0'], ['Movie name: Flashdance (1983)', 'Rating:6.0'], ['Movie name: Rocky III (1982)', 'Rating:8.0'], ['Movie name: Running Scared (1986)', 'Rating:8.0'], ['Movie name: Bloodsport (1988)', 'Rating:8.0'], ['Movie name: View to a Kill, A (1985)', 'Rating:6.0'], ['Movie name: Santa Claus: The Movie (1985)', 'Rating:6.0'], ['Movie name: Karate Kid, Part II, The (1986)', 'Rating:4.0'], ['Movie name: Rocky IV (1985)', 'Rating:8.0'], ['Movie name: Firestarter (1984)', 'Rating:6.0'], ['Movie name: Karate Kid III, The (1989)', 'Rating:2.0'], ['Movie name: Crocodile Dundee II (1988)', 'Rating:4.0'], ['Movie name: Superman III (1983)', 'Rating:2.0'], ['Movie name: Soul Man (1986)', 'Rating:4.0'], ['Movie name: Gung Ho (1986)', 'Rating:6.0'], ['Movie name: Police Academy 2: Their First Assignment (1985)', 'Rating:6.0'], ['Movie name: Toxic Avenger Part III: The Last Temptation of Toxie, The (1989)', 'Rating:6.0'], ['Movie name: Police A